# Login and Initialize
## Login Using Spotify
After running this click Sign-In and login to Spotify. **Note** you cannot run the whole Notebook through. Run cell #1 first.

In [ ]:
%load_ext autoreload
%autoreload 2
from ipyauth import ParamsSpotify, Auth
auth = Auth(ParamsSpotify(
    redirect_uri='http://localhost:8888/callback', 
    client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

## Fetch the Library
Initialize the Spotify object with a `logger` and fetch the library DataFrame. Check it works by outputting the Profile image.

In [ ]:
import requests
from IPython.display import Image
from spotify import Spotify
from pandas import DataFrame as df
from datetime import datetime,timedelta,date
import json
import logging

logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

spot = Spotify(auth, logger = logger)
lib = spot.fetchLibraryDataFrame( )
# Common Clauses
released1Y = (lib.track_released>(datetime.now()-timedelta(days=365)))
released4M = (lib.track_released>(datetime.now()-timedelta(days=120)))
released3M = (lib.track_released>(datetime.now()-timedelta(days=90)))
added2W = (lib.added_at>(datetime.now()-timedelta(days=14)).isoformat())
added3M = (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat())
added4M = (lib.added_at>(datetime.now()-timedelta(days=120)).isoformat())

# Debug playlists - used to exclude
noise = spot.fetchPlaylistUris("Noise")
decSongs = spot.fetchPlaylistUris("December Songs")
holidaySongs = spot.fetchPlaylistUris("Holiday Songs")

#lib.columns.values
Image(url=spot.imageUrl, width=50)

## Holiday Playlists



In [ ]:

allDecember = lib[(lib.original_uri.isin(decSongs)) | (lib.original_uri.isin(holidaySongs)) ]
#lib.columns.values
import pandas as pd

#allDecember.head(1)
lib[pd.isnull(lib.danceability)][["track_name","artist","original_uri","name","original_id"]]

spot.updatePlaylist("Auto December",allDecember)
spot.updatePlaylist("Auto Christmas Party", allDecember[ (allDecember.energy>0.7) & (allDecember.acousticness<0.5) ])
spot.updatePlaylist("Auto Holly Jolly", allDecember[ (allDecember.energy>0.6) & (allDecember.instrumentalness<0.9) & (allDecember.acousticness<0.7) ])
spot.updatePlaylist("Auto December Chill", allDecember[ (allDecember.energy<0.701) & (allDecember.instrumentalness>0.2) ])
spot.updatePlaylist("Auto Silent Night", allDecember[ (allDecember.danceability<0.4) & (allDecember.energy<0.4) & (allDecember.instrumentalness>0.4) ])
spot.updatePlaylist("Auto Work Christmas", allDecember[ (allDecember.instrumentalness>0.2) & (allDecember.energy>0.2) ])
spot.updatePlaylist("Auto Instrumental Christmas", allDecember[ allDecember.instrumentalness>0.75 ])
spot.updatePlaylist("Auto Added Christmas",allDecember[ allDecember.added_at>(datetime.now()-timedelta(days=7)).isoformat()])
spot.updatePlaylist("Auto Recent Christmas",allDecember[ allDecember.track_released>(datetime.now()-timedelta(days=365))])
spot.updatePlaylist("Auto Classic Christmas",allDecember[ allDecember.track_released<(datetime(1991,1,1))])

#lib.columns.values


In [ ]:
from pandas import DataFrame, read_pickle, merge
# tracks = read_pickle("mytracks.pkl")
#tracks[["track_id","track_name","track_artists"]].iloc[0:5]
#tracks[["track_id"]].iloc[0:1,0].values.tolist()
#[x for i,x in enumerate([True,False,True])]

#lib.head()
#ff = lib.drop(lib.index[x] for x in [0,1])
#ff.head()


#spot.checkDeletes( lib[["original_id"]].iloc[:,0].values.tolist() )

# Playlist Updates

## A Running Playlist

This playlist has high tempo and energy. The preference is for instrumentals with high danceability. Avoid long songs to keep the tracks changing.

In [ ]:
newPlaylist = lib[
    (lib.tempo>130) & 
    (lib.instrumentalness>0.3) & 
    (lib.duration_ms<1000*60*60) & 
    (lib.danceability>0.7) &
    (lib.track_released>datetime(1990,1,1))
].sort_values(["added_at"])
# Exclude some artists from this playlist
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Radiohead","Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist,excludingPlaylists=[holidaySongs])

## Unpopular or Hispter?
This playlists tracks tracks with low popularity. This is useful for checking if you added the most commonly played track (e.g. a version before it was remastered).

In [ ]:
#lib[lib.track_album_album_type=="compilation"]
spot.updatePlaylist("Are You Sure",lib[(lib.track_popularity<2) & (lib.track_is_playable==True)],excludingPlaylists=[holidaySongs,noise,"New Released"])
spot.updatePlaylist("Unplayable",lib[(lib.track_is_playable==False)],excludingPlaylists=[])
spot.updatePlaylist("Sinlges Compilations",lib[(lib.track_album_album_type=="compilation")],excludingPlaylists=[])


## Decades of Music
These playlists bucket the music in your library into decades using the release date. 

In [ ]:
spot.updatePlaylist("Auto 1960s",lib[lib.track_released<datetime(1970,1,1)],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1970s",lib[(lib.track_released>datetime(1970,1,1)) & (lib.track_released<datetime(1980,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1980s",lib[(lib.track_released>datetime(1980,1,1)) & (lib.track_released<datetime(1990,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1990s",lib[(lib.track_released>datetime(1990,1,1)) & (lib.track_released<datetime(2000,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2000s",lib[(lib.track_released>datetime(2000,1,1)) & (lib.track_released<datetime(2010,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2010s",lib[(lib.track_released>datetime(2010,1,1)) & (lib.track_released<datetime(2020,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Good Year",lib[released1Y],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2020",lib[lib.track_released>=datetime(2020,1,1)],excludingPlaylists=[holidaySongs])


## Recently Added
The music you added in the last couple of months.

In [ ]:
spot.updatePlaylist("Auto 2W",lib[added2W],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("Auto 3M",lib[added3M],excludingPlaylists=[holidaySongs,noise,"Auto 2W"])
spot.updatePlaylist("New Released",lib[released3M],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("New Mix Takes",lib[released3M & (lib.energy>0.3)],excludingPlaylists=[holidaySongs,"Auto 2W"])

## Mood Music
Music for every occasion. Low energy dinner parties, going to sleep or concentrating at work.

In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.loudness<-15) & (lib.speechiness<0.3) & (lib.energy<0.2) & (lib.danceability<0.3)],excludingPlaylists=[holidaySongs])

workInstrumental = ((lib.speechiness<0.08) & (lib.instrumentalness>0.87)) & (lib.energy<0.8)
workDance = (lib.danceability>0.3)
quietInstrumental = workInstrumental & (lib.energy<0.3)
quietDance = (lib.danceability<0.5)
quietEnergy = (lib.loudness<-15)
loudEnergy = (lib.loudness >= -15)
workFast = (lib.danceability>0.7) & ((lib.speechiness<0.2) & (lib.instrumentalness>0.67)) & (lib.energy>0.5) 

spot.updatePlaylist("To Work To",lib[workInstrumental & workDance ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("To Work To 4M",lib[workInstrumental & workDance & added4M],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("New Released To Work To",lib[workInstrumental & workDance & released1Y],excludingPlaylists=[holidaySongs])

spot.updatePlaylist("Quiet Work",lib[quietInstrumental & quietDance & quietEnergy ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Loud Work",lib[workInstrumental & workDance & loudEnergy],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Fast Work",lib[workFast],excludingPlaylists=[holidaySongs])


## More Analysis
It's easy to add the same track twice, from different albums. This can cause duplicates in playlists. We can use Pandas `groupby` to scan for that

In [ ]:
# Duplicates in the library
ff = lib.groupby(["track_name","artist","track_duration_ms"]).filter(lambda x : len(x)>1)
spot.updatePlaylist("Duplicates",ff,excludingPlaylists=[holidaySongs])

bestOf = lib[(lib.track_album_name.str.contains("Best Of")) | (lib.track_album_name.str.contains("Greatest")) | (lib.track_album_name.str.contains("The Essential"))]
spot.updatePlaylist("Suspect Best Of",bestOf,excludingPlaylists=[holidaySongs])

#lib[[lib.track_id.isin(ff["track_id"].values)]]

In [ ]:
lib.columns.values
lib[lib.track_album_name.isin(["The Essential Collection","Memorial Beach","25"])][["artist","track_name","speechiness","instrumentalness","danceability","energy","original_id"]]